## (년 -> 월/분기/반기) 데이터 변환

In [10]:
def fn_year_data_transform(file_nm, save_range, YEAR_COL_NM, COL1, COL2):
    import pandas as pd
    import copy
    import warnings
    warnings.filterwarnings('ignore')
    
    data = pd.read_csv(file_nm + '.csv', encoding = 'cp949', dtype = 'str')
    nujuk_yr_to_mon_data = pd.DataFrame()
    
    COL1_list = list(data[COL1].unique())
    COL2_list = list(data[COL2].unique())
    
    # ---------------------------------------------------------- #
    # 데이터 형 변환(str -> float)
    str_col = [COL1, COL2]
    int_col = list(set(data.columns).difference(set(str_col)))
    
    for chg_col in int_col:
        data[chg_col] = data[chg_col].astype('float')
    # ---------------------------------------------------------- #
    
    for col1 in COL1_list:
        for col2 in COL2_list:
            x = data.loc[(data[COL1] == col1) & (data[COL2] == col2),]
            x = x.sort_values(by = YEAR_COL_NM).reset_index(drop=True)

            col_list = list(x.columns)
            mody_list = list(x.columns)
            remove_list = [YEAR_COL_NM,COL1,COL2]

            for i in remove_list:
                mody_list.remove(i) 
                
            for mody_col in mody_list:
                df = x[[YEAR_COL_NM, mody_col]]
                tmp_df = x[[mody_col]].shift(1).rename(columns = {mody_col:str(mody_col+'_sft')})
                df = pd.concat([df, tmp_df], axis = 1)

                df['diff'] = df[mody_col] - df[mody_col+'_sft']
                df['diff_12'] = df['diff']/12
                df['diff_12'] = df['diff_12'].shift(-1)
                df = df[[YEAR_COL_NM, mody_col, 'diff_12']]

                # num1 : df의 개수
                # num2 : 1/12값 곱해줄 month
                nujuk_mon_val = []  # 1/12값 곱해서 append할 장소
                nujuk_year = []
                nujuk_month = []

                for num1 in range(len(df)):
                    for num2 in range(0,12):
                        if num2 == 0:
                            # nujuk_mon_val : 생성한 월값
                            nujuk_mon_val.append(df[mody_col][num1])   
                            # nujuk_year : 연도
                            nujuk_year.append(df[YEAR_COL_NM][num1])
                            # nujuk_month : 월
                            nujuk_month.append(12)  
                        else:
                            if num1 != (len(df) - 1):
                                # nujuk_mon_val : 생성한 월값
                                nujuk_mon_val.append(df[mody_col][num1] + (num2 * df['diff_12'][num1]))
                                # nujuk_year : 연도
                                nujuk_year.append(df[YEAR_COL_NM][num1] + 1)
                                # nujuk_month : 월
                                nujuk_month.append((num2))  

                mon_val_df = pd.DataFrame(nujuk_mon_val).rename(columns = {0:mody_col})
                year_df = pd.DataFrame(nujuk_year).rename(columns = {0:'YEAR'})
                month_df = pd.DataFrame(nujuk_month).rename(columns = {0:'MONTH'})

                if mody_col == mody_list[0]:
                    yr_to_mon_data = pd.concat([year_df, month_df, mon_val_df], axis = 1)
                else:
                    yr_to_mon_data = pd.concat([yr_to_mon_data, mon_val_df], axis = 1)
            
            # 컬럼 순서 변경
            basic_col = [COL1,COL2] + list(yr_to_mon_data.columns)            
            yr_to_mon_data[COL1] = col1
            yr_to_mon_data[COL2] = col2
            yr_to_mon_data = yr_to_mon_data[basic_col]

            # 데이터 누적
            nujuk_yr_to_mon_data = nujuk_yr_to_mon_data.append(yr_to_mon_data)
            # print('nujuk_yr_to_mon_data 건수 : ', len(nujuk_yr_to_mon_data))

    ## nujuk_yr_to_mon_data : 월데이터
    
    # MONTH 컬럼 수정
    def fn_mody_month(x):
        if int(x) < 10:
            x = str('0' + str(x))
        return x

    nujuk_yr_to_mon_data['MONTH'] = list(map(fn_mody_month, nujuk_yr_to_mon_data['MONTH']))
    nujuk_yr_to_mon_data['MONTH'] = nujuk_yr_to_mon_data['MONTH'].astype('str')
    nujuk_yr_to_mon_data['YEAR'] = nujuk_yr_to_mon_data['YEAR'].astype('int').astype('str')
    
    # *************************************************************************************** #
    # 연 데이터 -> 월 데이터 저장
    month_data = copy.deepcopy(nujuk_yr_to_mon_data)
    # month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'cp949')
    # *************************************************************************************** #
    # 월 데이터 -> 분기 데이터 저장
    quarter_data = copy.deepcopy(nujuk_yr_to_mon_data)

    col_list = list(quarter_data.columns)
    mody_list = list(quarter_data.columns)
    remove_list = [COL1,COL2,'YEAR','MONTH']

    for i in remove_list:
        mody_list.remove(i)

        
    quarter_data.loc[quarter_data['MONTH'].isin(['01','02','04','05','07','08','10','11']),'QUARTER'] = '00'        
    quarter_data.loc[quarter_data['MONTH'].isin(['03']),'QUARTER'] = '01'
    quarter_data.loc[quarter_data['MONTH'].isin(['06']),'QUARTER'] = '02'
    quarter_data.loc[quarter_data['MONTH'].isin(['09']),'QUARTER'] = '03'
    quarter_data.loc[quarter_data['MONTH'].isin(['12']),'QUARTER'] = '04'

    basic_col = [COL1,COL2,'YEAR','QUARTER'] + mody_list

    quarter_data = quarter_data.groupby([COL1,COL2,'YEAR','QUARTER']).sum().reset_index()
    quarter_data = quarter_data[basic_col]
    quarter_data = quarter_data.loc[quarter_data['QUARTER'] != '00',]
    # quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'cp949')
    # *************************************************************************************** #
    # 월 데이터 -> 반기 데이터 저장
    half_data = copy.deepcopy(nujuk_yr_to_mon_data)

    col_list = list(half_data.columns)
    mody_list = list(half_data.columns)
    remove_list = [COL1,COL2,'YEAR','MONTH']

    for i in remove_list:
        mody_list.remove(i)
        
    half_data.loc[half_data['MONTH'].isin(['01','02','03','04','05','07','08','09','10','11']),'HALF'] = '00'
    half_data.loc[half_data['MONTH'].isin(['06']),'HALF'] = '01'
    half_data.loc[half_data['MONTH'].isin(['12']),'HALF'] = '02'

    basic_col = [COL1,COL2,'YEAR','HALF'] + mody_list

    half_data = half_data.groupby([COL1,COL2,'YEAR','HALF']).sum().reset_index()
    half_data = half_data[basic_col]
    half_data = half_data.loc[half_data['HALF'] != '00',]
    # half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'cp949')
    # *************************************************************************************** #
    # 결과 저장
    if save_range == 'all':
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'cp949')
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'cp949')
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'cp949')
    elif save_range == 'month':
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'cp949')
    elif save_range == 'quarter':
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'cp949')
    elif save_range == 'half':
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'cp949')
    else:
        print('월/분기/반기 모두 저장합니다.')
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'cp949')
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'cp949')
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'cp949')
    # *************************************************************************************** #
    return 'end'

In [11]:
import pandas as pd
import glob
file_list = glob.glob('data/*.csv')
file_list = pd.DataFrame(file_list).rename(columns = {0:'col'})
file_list = list(file_list['col'].apply(lambda x : x[:-4]))
file_list

['data/merge_age', 'data/merge_dis', 'data/merge_cls']

In [12]:
for file_nm in file_list:
    print(file_nm)
    tmp = pd.read_csv(file_nm + '.csv', encoding = 'cp949')
    col_list = list(tmp.columns[0:3])
    col = list(set(col_list).difference(set(['BASE_YY'])))
#     print(col)
    fn_year_data_transform(file_nm,'all','BASE_YY',col[0],col[1])  # fn_year_data_transform(file_nm,save_range,YEAR_COL_NM,COL1,COL2)
#     print(col[0],col[1])

data/merge_age
data/merge_dis
data/merge_cls


In [20]:
# fn_year_data_transform(file_list[0],'all','BASE_YY','SIDO','AGE')  # fn_year_data_transform(file_nm,save_range,YEAR_COL_NM,COL1,COL2)

In [21]:
# fn_year_data_transform(file_list[1],'all','BASE_YY','SIDO','DIS_TYPE')  # fn_year_data_transform(file_nm,save_range,YEAR_COL_NM,COL1,COL2)

In [22]:
# fn_year_data_transform(file_list[2],'all','BASE_YY','SIDO','CLS_NM')  # fn_year_data_transform(file_nm,save_range,YEAR_COL_NM,COL1,COL2)

--------

--------

--------

In [5]:
import pandas as pd
import glob
file_list = glob.glob('data/*.csv')
file_list = pd.DataFrame(file_list).rename(columns = {0:'col'})
file_list = list(file_list['col'].apply(lambda x : x[:-4]))
file_list

['data/merge_dis_month',
 'data/merge_age_half',
 'data/merge_age_month',
 'data/merge_age_quarter',
 'data/merge_dis_quarter',
 'data/merge_age',
 'data/merge_dis',
 'data/merge_cls',
 'data/merge_cls_quarter',
 'data/merge_dis_half',
 'data/merge_cls_half',
 'data/merge_cls_month']

In [13]:
data = pd.read_csv('data/merge_age_month' + '.csv', encoding = 'cp949', dtype = 'str')
data

,AGE,SIDO,YEAR,MONTH,SEP_CNT,DIS_CNT,EXAM_18_24,EXAM_30_36,EXAM_42_48,EXAM_4_6,...,EX_SUV_046,EX_SUV_048,EX_SUV_050,EX_SUV_054,EX_SUV_056,EX_SUV_058,EX_SUV_060,GRDT_KIN_CNT,GRDT_ELM_CNT,GRDT_MID_CNT
0,고1,강원,2010,12,215.0,237.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,31.0,262.0,217.0
1,고1,강원,2011,01,217.66666666666666,239.5,424.71799457509906,558.6907017759731,653.1685886528271,309.5954752829753,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,31.916666666666668,262.3333333333333,217.66666666666666
2,고1,강원,2011,02,220.33333333333334,242.0,426.83454052281735,560.3551834327028,651.6078078662065,314.359522984523,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,32.833333333333336,262.6666666666667,218.33333333333334
3,고1,강원,2011,03,223.0,244.5,428.95108647053564,562.0196650894325,650.0470270795859,319.1235706860707,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,33.75,263.0,219.0
4,고1,강원,2011,04,225.66666666666666,247.0,431.0676324182539,563.6841467461622,648.4862462929651,323.8876183876184,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,34.666666666666664,263.3333333333333,219.66666666666666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33370,초6,세종,2021,08,42.0,30.666666666666664,327.3413744740533,214.58399576046634,273.3151087595532,247.52234993614303,...,3.6201600886677405,1.0153215487117517,1.7366424505671159,20.03229635761983,79.96770364238017,0.0,0.0,36.333333333333336,30.0,35.666666666666664
33371,초6,세종,2021,09,43.75,31.75,318.75904628330994,195.28199523052461,257.47949735449737,238.21264367816093,...,3.5353057110145754,1.0152820429616911,1.6662274589139066,20.321519565512173,79.67848043448784,0.0,0.0,35.75,29.75,34.5
33372,초6,세종,2021,10,45.5,32.83333333333333,310.1767180925666,175.97999470058292,241.6438859494415,228.9029374201788,...,3.45045133336141,1.0152425372116305,1.5958124672606977,20.61074277340451,79.3892572265955,0.0,0.0,35.166666666666664,29.5,33.33333333333333
33373,초6,세종,2021,11,47.25,33.916666666666664,301.59438990182326,156.67799417064123,225.80827454438565,219.59323116219667,...,3.365596955708245,1.01520303146157,1.5253974756074884,20.89996598129685,79.10003401870316,0.0,0.0,34.583333333333336,29.25,32.166666666666664


In [14]:
data = pd.read_csv('data/merge_age' + '.csv', encoding = 'cp949', dtype = 'str')
data

,BASE_YY,SIDO,AGE,SEP_CNT,DIS_CNT,EXAM_18_24,EXAM_30_36,EXAM_42_48,EXAM_4_6,EXAM_54_60,...,EX_SUV_046,EX_SUV_048,EX_SUV_050,EX_SUV_054,EX_SUV_056,EX_SUV_058,EX_SUV_060,GRDT_KIN_CNT,GRDT_ELM_CNT,GRDT_MID_CNT
0,2010,강원,고1,215,237.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,715.3756119900992,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,31.0,262.0,217.0
1,2010,강원,고2,196,217.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,715.3756119900992,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,31.0,262.0,217.0
2,2010,강원,고3,173,219.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,715.3756119900992,...,31.86602870813397,5.516062884483937,14.716336295283664,10.808531148331124,82.04602964212556,4.590914568020244,2.554524641523075,31.0,262.0,217.0
3,2010,강원,유3,10,52.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,715.3756119900992,...,15.327009485771342,33.150274588117824,9.885172241637544,89.24050632911393,10.759493670886076,0.0,0.0,31.0,262.0,217.0
4,2010,강원,유4,24,74.0,422.6014486273808,557.0262201192434,654.7293694394477,304.8314275814276,715.3756119900992,...,15.327009485771342,33.150274588117824,9.885172241637544,89.24050632911393,10.759493670886076,0.0,0.0,31.0,262.0,217.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,2021,충북,초2,248,201.0,1270.0,1427.0,1408.0,821.0,1504.0,...,3.28074257805508,1.0151635257115095,1.4549824839542793,21.18918918918919,78.81081081081082,0.0,0.0,128.0,268.0,295.0
3011,2021,충북,초3,341,223.0,1270.0,1427.0,1408.0,821.0,1504.0,...,3.28074257805508,1.0151635257115095,1.4549824839542793,21.18918918918919,78.81081081081082,0.0,0.0,128.0,268.0,295.0
3012,2021,충북,초4,372,206.0,1270.0,1427.0,1408.0,821.0,1504.0,...,3.28074257805508,1.0151635257115095,1.4549824839542793,21.18918918918919,78.81081081081082,0.0,0.0,128.0,268.0,295.0
3013,2021,충북,초5,337,169.0,1270.0,1427.0,1408.0,821.0,1504.0,...,3.28074257805508,1.0151635257115095,1.4549824839542793,21.18918918918919,78.81081081081082,0.0,0.0,128.0,268.0,295.0
